In [1]:
#%pip install --force-reinstall -q "airavata-python-sdk[notebook]"
import airavata_jupyter_magic


Loaded airavata_jupyter_magic (2.0.12) 
(current runtime = local)

  %authenticate                      -- Authenticate to access high-performance runtimes.
  %request_runtime <rt> [args]       -- Request a runtime named <rt> with configuration <args>. Call multiple times to request multiple runtimes.
  %restart_runtime <rt>              -- Restart runtime <rt>. Run this if you install new dependencies or if the runtime hangs.
  %stop_runtime <rt>                 -- Stop runtime <rt> when no longer needed.
  %switch_runtime <rt>               -- Switch active runtime to <rt>. All subsequent executions will use this runtime.
  %%run_on <rt>                      -- Force a cell to always execute on <rt>, regardless of the active runtime.
  %stat_runtime <rt>                 -- Show the status of runtime <rt>.
  %copy_data <r1:file1> <r2:file2>   -- Copy <file1> in <r1> to <file2> in <r2>.



In [3]:
%authenticate

Output()

Authenticated.

In [4]:
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=expanse:shared
%switch_runtime hpc_cpu

Requesting runtime=hpc_cpu
cpuCount: 4
experimentName: CS_Agent
group: Default
libraries:
- numpy=2.2.5
- pandas=2.2.3
- pip
- psycopg2=2.9.9
- postgresql=17.4
- pytest
- python=3.10
- psi4=1.9.1
- pytorch-cpu=2.5.1
- jupyter=1.1.1
- requests
- setuptools
- torchaudio=2.5.1
- torchvision=0.20.1
- pytorch_geometric=2.6.1
- pytorch_scatter=2.1.2=cpu*
- pytorch-minimize=0.0.2
- matplotlib=3.10.1
- pydantic=1
- scipy=1.15.*
- tqdm
memory: 8000
mounts:
- cybershuttle-reference:/cybershuttle_data/cybershuttle-reference
nodeCount: 1
pip:
- cdsg-tools==0.0.2
- qm-tools-aw==1.4.5
- qcmlforge==0.0.6
- qcfractal==0.59
- qcmanybody
- qcfractalcompute==0.59
queue: shared
remoteCluster: expanse
wallTime: 60

Requested runtime=hpc_cpu. state=CONFIGURING_WORKSPACE
Switched to runtime=hpc_cpu.


In [19]:
%ls
%cd /workspace
%ls
%copy_data source=local:setup_qcfractal.py target=hpc_cpu:setup_qcfractal.py
%copy_data source=local:__init__.py target=hpc_cpu:__init__.py

apnet2_transfer_learning.pt  __init__.py            qcfractal_server.log
combined_df_subset_358.pkl   LICENSE                README.md
cybershuttle.yml             main.py                S22-IE-AP2-dAP2_violin.png
data_dir/                    notes.md               S22-IE-AP2_violin.png
demo.html                    og_qcfractal/          S22-IE_violin.png
demo.ipynb                   parsl_run_dir/         s22.py
demo_og.py                   __pycache__/           setup_qcfractal.py
demo.py                      qcfractal/             timer.dat
environment.yml              qcfractal_compute.log  tmp/
[Errno 2] No such file or directory: '/workspace'
/home/amwalla3/gits/cybershuttle_demo
apnet2_transfer_learning.pt  __init__.py            qcfractal_server.log
combined_df_subset_358.pkl   LICENSE                README.md
cybershuttle.yml             main.py                S22-IE-AP2-dAP2_violin.png
data_dir/                    notes.md               S22-IE-AP2_violin.png
demo.html        

# QCArchive+QCMLForge Demo with CyberShuttle

The first half of this demo shows how to use QCArchive to setup a dataset
and run computations with ease. The compute resource can be local or
through CyberShuttle. 

The second half of this demo shows how one can consume the generated data
to train AP-Net2 and dAPNet2 models through QCMLForge. 

In [6]:
import psi4
from pprint import pprint as pp
import pandas as pd
import numpy as np
import re
from qm_tools_aw import tools
from pprint import pprint as pp
# QCElemental Imports
from qcelemental.models import Molecule
import qcelemental as qcel
# Dataset Imports
from qcportal import PortalClient
from qcportal.singlepoint import SinglepointDataset, SinglepointDatasetEntry, QCSpecification
from qcportal.manybody import ManybodyDataset, ManybodyDatasetEntry, ManybodyDatasetSpecification, ManybodySpecification
from torch import manual_seed

manual_seed(42)

h2kcalmol = qcel.constants.hartree2kcalmol
print('Imports')

⠼ Connecting to=hpc_cpu... status=CONNECTEDWORKSPACE
Imports


# QCArchive Setup

In [ ]:
from setup_qcfractal import setup_qcarchive_qcfractal
import os

setup_qcarchive_qcfractal(
    QCF_BASE_FOLDER=os.path.join(os.getcwd(), "qcfractal"),
    start=False,
    reset=False,
    db_config={
        "name": None,
        "enable_security": "false",
        "allow_unauthenticated_read": None,
        "logfile": None,
        "loglevel": None,
        "service_frequency": 5,
        "max_active_services": None,
        "heartbeat_frequency": 60,
        "log_access": None,
        "database": {
            "base_folder": None,
            "host": None,
            "port": 5433,
            "database_name": "qca",
            "username": None,
            "password": None,
            "own": None,
        },
        "api": {
            "host": None,
            "port": 7778,
            "secret_key": None,
            "jwt_secret_key": None,
        },
    },
    resources_config={
            "update_frequency": 5,
            "cores_per_worker": 8,
            "max_workers": 3,
            "memory_per_worker": 20,
    },
    conda_env=False,
)

⠹ Connecting to=hpc_cpu... status=CONNECTED
/workspace/qcfractal
--------------------------------------------------------------------------------
Python executable:  /dev/shm/cybershuttle/envs/18dccede/bin/python3.10
QCFractal version:  0.59
QCFractal alembic revision:  d5988aa750ae
pg_ctl path:  /dev/shm/cybershuttle/envs/18dccede/bin/pg_ctl
PostgreSQL server version:  PostgreSQL 17.4 on x86_64-conda-linux-gnu, compiled by x86_64-conda-linux-gnu-cc (conda-forge gcc 13.3.0-2) 13.3.0, 64-bit
--------------------------------------------------------------------------------


Displaying QCFractal configuration below
--------------------------------------------------------------------------------
access_log_keep: 0
allow_unauthenticated_read: true
api:
  extra_flask_options: null
  extra_waitress_options: null
  host: localhost
  jwt_access_token_expires: 3600
  jwt_refresh_token_expires: 86400
  jwt_secret_key: UCTKvPSYoYq-hyxLFHMcXm2ZeHeYtcjYMRj4_uHCcWc
  num_threads_per_worker: 4
  port:

In [10]:
%%bash
pg_ctl --version
ls /workspace/qcfractal
#pg_ctl -D /workspace/qcfractal/postgres stop -m immediate

⠹ Connecting to=hpc_cpu... status=CONNECTED
pg_ctl (PostgreSQL) 17.4
postgres
qcfractal_config.yaml
qcfractal_database.log


In [12]:
get_ipython().system = os.system
!qcfractal-server --config=`pwd`/qcfractal/qcfractal_config.yaml start &

⠼ Connecting to=hpc_cpu... status=CONNECTED
0


In [13]:
get_ipython().system = os.system
!qcfractal-compute-manager --config=`pwd`/qcfractal/resources.yml &

⠹ Connecting to=hpc_cpu... status=CONNECTED
[2025-04-30 16:51:55 PDT] (MainProcess     )     INFO: qcfractal.qcfractal_server_cli: Checking the PostgreSQL connection...
[2025-04-30 16:51:55 PDT] (MainProcess     )     INFO: PostgresHarness: /dev/shm/cybershuttle/envs/18dccede/bin

[2025-04-30 16:51:55 PDT] (MainProcess     )     INFO: PostgresHarness: Using Postgres tools found via pg_config located in /dev/shm/cybershuttle/envs/18dccede/bin
[2025-04-30 16:51:55 PDT] (MainProcess     )     INFO: PostgresHarness: pg_ctl (PostgreSQL) 17.4

[2025-04-30 16:51:55 PDT] (MainProcess     )     INFO: PostgresHarness: Postgresql version found: pg_ctl (PostgreSQL) 17.4
[2025-04-30 16:51:55 PDT] (MainProcess     )     INFO: PostgresHarness: Starting the PostgreSQL instance
[2025-04-30 16:51:57 PDT] (MainProcess     )     INFO: PostgresHarness: waiting for server to start..... done
server started

[2025-04-30 16:51:57 PDT] (MainProcess     )     INFO: PostgresHarness: pg_ctl: server is running (PID

In [18]:
!conda 

⠹ Connecting to=hpc_cpu... status=CONNECTED


32512


In [ ]:
# NOTE kill server when finished by removing the # and executing:
# !ps aux | grep qcfractal | awk '{ print $2 }' | xargs kill -9

In [ ]:
!qcfractal-server --config=`pwd`/qcfractal/qcfractal_config.yaml init-db
!qcfractal-server --config=`pwd`/qcfractal/qcfractal_config.yaml --help
!qcfractal-server --config=`pwd`/qcfractal/qcfractal_config.yaml start

In [57]:
%%bash
for pid in /proc/[0-9]*; do
    echo "${pid##*/}" `cat ${pid}/comm` > procs.txt
done

⠼ Connecting to=hpc_cpu... status=CONNECTED


In [59]:
!grep -e 'postgres' ./procs.txt
!grep -e 'qcfractal' ./procs.txt

⠹ Connecting to=hpc_cpu... status=CONNECTED
256


In [ ]:
#!grep -e 'postgres' ./procs.txt | awk '{ print $1 }' | xargs kill -9

⠏ Connecting to=hpc_cpu... status=CONNECTED
xargs: kill: No such file or directory


In [14]:
!ls qcfractal/
!cat ./qcfractal/qcfractal_database.log

⠸ Connecting to=hpc_cpu... status=CONNECTED
[2025-04-30 16:52:00 PDT]     INFO: qcfractalcompute.config: Reading configuration data from /workspace/qcfractal/resources.yml
**********
Logging to file /workspace/qcfractal/qcfractal-manager.log, logging level INFO
**********


[2025-04-30 16:52:03 PDT] (Process-2       )     INFO: internal_job_runner:4bb8e226-98dd-4473-81f9-3f668e5e985a: running job iterate_services id=5 scheduled_date=2025-04-30 16:52:03.414367-07:00
[2025-04-30 16:52:03 PDT] (Process-2       )     INFO: qcfractal.components.services.socket: Iterating on services
[2025-04-30 16:52:03 PDT] (Process-2       )     INFO: qcfractal.components.services.socket: After iteration, now 0 running services. Max is 20
[2025-04-30 16:52:08 PDT] (Process-2       )     INFO: internal_job_runner:4bb8e226-98dd-4473-81f9-3f668e5e985a: running job iterate_services id=7 scheduled_date=2025-04-30 16:52:08.695165-07:00
[2025-04-30 16:52:08 PDT] (Process-2       )     INFO: qcfractal.components.services.socket: Iterating on services
[2025-04-30 16:52:08 PDT] (Process-2       )     INFO: qcfractal.components.services.socket: After iteration, now 0 running services. Max is 20
geoip2
postgres
qcfractal-manager.log
qcfractal_config.yaml
qcfractal_database.log
resources

# QCArchive single point example

In [64]:
# Establish client connection
client = PortalClient("http://localhost:7778", verify=False)

⠋ Connecting to=hpc_cpu... status=CONNECTED
[2025-04-30 14:50:40 PDT]     INFO: qcfractalcompute.config: Reading configuration data from /workspace/qcfractal/resources.yml
**********
Logging to file /workspace/qcfractal/qcfractal-manager.log, logging level INFO
**********


[2025-04-30 14:50:43 PDT] (Process-2       )     INFO: internal_job_runner:55b42e18-773c-438f-93cc-5176b8df6e79: running job iterate_services id=7 scheduled_date=2025-04-30 14:50:43.106314-07:00
[2025-04-30 14:50:43 PDT] (Process-2       )     INFO: qcfractal.components.services.socket: Iterating on services
[2025-04-30 14:50:43 PDT] (Process-2       )     INFO: qcfractal.components.services.socket: After iteration, now 0 running services. Max is 20
[2025-04-30 14:50:49 PDT] (Process-2       )     INFO: internal_job_runner:55b42e18-773c-438f-93cc-5176b8df6e79: running job iterate_services id=8 scheduled_date=2025-04-30 14:50:49.051582-07:00
[2025-04-30 14:50:49 PDT] (Process-2       )     INFO: qcfractal.components.services.socket: Iterating on services
[2025-04-30 14:50:49 PDT] (Process-2       )     INFO: qcfractal.components.services.socket: After iteration, now 0 running services. Max is 20
[2025-04-30 14:50:54 PDT] (Process-2       )     INFO: internal_job_runner:55b42e18-773c-438

In [50]:
# Running a single job
client = PortalClient("http://localhost:7778", verify=False)
mol = Molecule.from_data(
    """
     0 1
     O  -1.551007  -0.114520   0.000000
     H  -1.934259   0.762503   0.000000
     H  -0.599677   0.040712   0.000000
     --
     0 1
     O   1.350625   0.111469   0.000000
     H   1.680398  -0.373741  -0.758561
     H   1.680398  -0.373741   0.758561

     units angstrom
     no_reorient
     symmetry c1
"""
)

psi4.set_options(
    {"basis": "aug-cc-pvdz", "scf_type": "df", "e_convergence": 6, "freeze_core": True}
)

client.add_singlepoints(
    [mol],
    "psi4",
    driver="energy",
    method="b3lyp",
    basis="aug-cc-pvdz",
    keywords={"scf_type": "df", "e_convergence": 6, "freeze_core": True},
    tag="local",
)

# Can print records
# for rec in client.query_records():
#     pp(rec.dict)
#     pp(rec.error)

⠸ Connecting to=hpc_cpu... status=CONNECTED
[2025-04-30 14:44:40 PDT] (MainProcess     )     INFO: PostgresHarness: waiting for server to start............................................................... stopped waiting

[2025-04-30 14:44:40 PDT] (MainProcess     )     INFO: PostgresHarness: pg_ctl: server did not start in time


# QCArchive dataset examples

In [ ]:
# Creating a QCArchive Dataset...
# Load in a dataset from a recent Sherrill work (Levels of SAPT II)
df_LoS = pd.read_pickle("./combined_df_subset_358.pkl")
print(df_LoS[['Benchmark', 'SAPT2+3(CCD)DMP2 TOTAL ENERGY aqz', 'MP2 IE atz', 'SAPT0 TOTAL ENERGY adz' ]])

# Limit to 100 molecules with maximum of 16 atoms to keep computational cost down
df_LoS['size'] = df_LoS['atomic_numbers'].apply(lambda x: len(x))
df_LoS = df_LoS[df_LoS['size'] <= 16]
df_LoS = df_LoS.sample(200, random_state=42, axis=0).copy()
df_LoS.reset_index(drop=True, inplace=True)
print(df_LoS['size'].describe())

# Create QCElemntal Molecules to generate the dataset
def qcel_mols(row):
    """
    Convert the row to a qcel molecule
    """
    atomic_numbers = [row['atomic_numbers'][row['monAs']], row['atomic_numbers'][row['monBs']]]
    coords = [row['coordinates'][row['monAs']], row['coordinates'][row['monBs']]]
    cm = [
        [row['monA_charge'], row['monA_multiplicity']],
        [row['monB_charge'], row['monB_multiplicity']],
     ]
    return tools.convert_pos_carts_to_mol(atomic_numbers, coords, cm)
df_LoS['qcel_molecule'] = df_LoS.apply(qcel_mols, axis=1)
geoms = df_LoS['qcel_molecule'].tolist()
ref_IEs = df_LoS['Benchmark'].tolist()
sapt0_adz = (df_LoS['SAPT0 TOTAL ENERGY adz'] * h2kcalmol).tolist()

## Singlepoint Dataset

In [ ]:
# Create client dataset

ds_name = 'S22-singlepoint'
client_datasets = [i['dataset_name'] for i in client.list_datasets()]
# Check if dataset already exists, if not create a new one
if ds_name not in client_datasets:
    ds = client.add_dataset("singlepoint", ds_name,
                            f"Dataset to contain {ds_name}")
    print(f"Added {ds_name} as dataset")
    # Insert entries into dataset
    entry_list = []
    for idx, mol in enumerate(geoms):
        extras = {
            "name": 'S22-' + str(idx),
            "idx": idx,
        }
        mol = Molecule.from_data(mol.dict(), extras=extras)
        ent = SinglepointDatasetEntry(name=extras['name'], molecule=mol)
        entry_list.append(ent)
    ds.add_entries(entry_list)
    print(f"Added {len(entry_list)} molecules to dataset")
else:
    ds = client.get_dataset("singlepoint", ds_name)
    print(f"Found {ds_name} dataset, using this instead")

print(ds)

In [ ]:
# Can delete the dataset if you want to start over. Need to know dataset_id
# client.delete_dataset(dataset_id=ds.id, delete_records=True)

In [ ]:
# Multipole Example
# method, basis = "hf", "sto-3g"
#
# # Set the QCSpecification (QM interaction energy in our case)
# spec = QCSpecification(
#     program="psi4",
#     driver="energy",
#     method=method,
#     basis=basis,
#     keywords={
#         "d_convergence": 8,
#         "dft_radial_points": 99,
#         "dft_spherical_points": 590,
#         "e_convergence": 10,
#         "guess": "sad",
#         "mbis_d_convergence": 9,
#         "mbis_radial_points": 99,
#         "mbis_spherical_points": 590,
#         "scf_properties": ["mbis_charges", "MBIS_VOLUME_RATIOS"],
#         "scf_type": "df",
#     },
#     protocols={"wavefunction": "orbitals_and_eigenvalues"},
# )
# ds.add_specification(name=f"psi4/{method}/{basis}", specification=spec)

In [ ]:
# SAPT0 Example
method, basis = "SAPT0", "cc-pvdz"

# Set the QCSpecification (QM interaction energy in our case)
spec = QCSpecification(
    program="psi4",
    driver="energy",
    method=method,
    basis=basis,
    keywords={
        "scf_type": "df",
    },
)
ds.add_specification(name=f"psi4/{method}/{basis}", specification=spec)

In [ ]:
# Run the computations
ds.submit()
print(f"Submitted {ds_name} dataset")

In [ ]:
# Check the status of the dataset - can repeatedly run this to see the progress
ds.status()

## Manybody Dataset

In [ ]:
# Create client dataset
ds_name_mb = 'S22-manybody'
client_datasets = [i['dataset_name'] for i in client.list_datasets()]
# Check if dataset already exists, if not create a new one
if ds_name_mb not in client_datasets:
    print("Setting up new dataset:", ds_name_mb)
    ds_mb = client.add_dataset("manybody", ds_name_mb,
                            f"Dataset to contain {ds_name_mb}")
    print(f"Added {ds_name_mb} as dataset")
    # Insert entries into dataset
    entry_list = []
    for idx, mol in enumerate(geoms):
        ent = ManybodyDatasetEntry(name=f"S22-IE-{idx}", initial_molecule=mol)
        entry_list.append(ent)
    ds_mb.add_entries(entry_list)
    print(f"Added {len(entry_list)} molecules to dataset")
else:
    ds_mb = client.get_dataset("manybody", ds_name_mb)
    print(f"Found {ds_name_mb} dataset, using this instead")

print(ds_mb)

# Can delete the dataset if you want to start over. Need to know dataset_id
# client.delete_dataset(dataset_id=2, delete_records=True)

In [ ]:
ds_mb.status()

In [ ]:
# Set multiple levels of theory - you can add/remove levels as you desire.
# Computational scaling will get quite expensive with better methods and larger
# basis sets

methods = [
    'hf', 'pbe',
]
basis_sets = [
    '6-31g*'
]

for method in methods:
    for basis in basis_sets:
        # Set the QCSpecification (QM interaction energy in our case)
        qc_spec_mb = QCSpecification(
            program="psi4",
            driver="energy",
            method=method,
            basis=basis,
            keywords={
                "d_convergence": 8,
                "scf_type": "df",
            },
        )

        spec_mb = ManybodySpecification(
            program='qcmanybody',
            bsse_correction=['cp'],
            levels={
                1: qc_spec_mb,
                2: qc_spec_mb,
            },
        )
        print("spec_mb", spec_mb)

        ds_mb.add_specification(name=f"psi4/{method}/{basis}", specification=spec_mb)

        # Run the computations
        ds_mb.submit()
        print(f"Submitted {ds_name} dataset")
# Check the status of the dataset - can repeatedly run this to see the progress
ds_mb.status()

In [ ]:
pp(ds.status())
pp(ds_mb.status())

In [ ]:
pp(ds)
pp(ds_mb)
pp(ds_mb.computed_properties)

# Data Assembly

While you can execute the following blocks before all computations are complete, it is recommended to wait until all computations are complete to continue.

In [ ]:
# Singlepoint data assemble
def assemble_singlepoint_data(record):
    record_dict = record.dict()
    qcvars = record_dict["properties"]
    level_of_theory = f"{record_dict['specification']['method']}/{record_dict['specification']['basis']}"
    sapt_energies = np.array([np.nan, np.nan, np.nan, np.nan, np.nan])
    if "mbis charges" in qcvars:
        charges = qcvars["mbis charges"]
        dipoles = qcvars["mbis dipoles"]
        quadrupoles = qcvars["mbis quadrupoles"]
        n = len(charges)
        charges = np.reshape(charges, (n, 1))
        dipoles = np.reshape(dipoles, (n, 3))
        quad = np.reshape(quadrupoles, (n, 3, 3))

        quad = [q[np.triu_indices(3)] for q in quad]
        quadrupoles = np.array(quad)
        multipoles = np.concatenate(
            [charges, dipoles, quadrupoles], axis=1)
        return (
        record.molecule,
        qcvars['mbis volume ratios'],
        qcvars['mbis valence widths'],
        qcvars['mbis radial moments <r^2>'],
        qcvars['mbis radial moments <r^3>'],
        qcvars['mbis radial moments <r^4>'],
        record.molecule.atomic_numbers,
        record.molecule.geometry * qcel.constants.bohr2angstroms,
        multipoles,
        int(record.molecule.molecular_charge),
        record.molecule.molecular_multiplicity,
        sapt_energies,
        )
    else:
        sapt_energies[0] = qcvars['sapt total energy']
        sapt_energies[1] = qcvars['sapt elst energy']
        sapt_energies[2] = qcvars['sapt exch energy']
        sapt_energies[3] = qcvars['sapt ind energy']
        sapt_energies[4] = qcvars['sapt disp energy']
        return (
        record.molecule,
        None,
        None,
        None,
        None,
        None,
        record.molecule.atomic_numbers,
        record.molecule.geometry * qcel.constants.bohr2angstroms,
        None,
        int(record.molecule.molecular_charge),
        record.molecule.molecular_multiplicity,
        sapt_energies,
        )




def assemble_singlepoint_data_value_names():
    return [
        'qcel_molecule',
        "volume ratios",
        "valence widths",
        "radial moments <r^2>",
        "radial moments <r^3>",
        "radial moments <r^4>",
        "Z",
        "R",
        "cartesian_multipoles",
        "TQ",
        "molecular_multiplicity",
        "SAPT Energies"
    ]

df = ds.compile_values(
    value_call=assemble_singlepoint_data,
    value_names=assemble_singlepoint_data_value_names(),
    unpack=True,
)
pp(df.columns.tolist())
df_sapt0 = df['psi4/SAPT0/cc-pvdz']

In [ ]:
def assemble_data(record):
    record_dict = record.dict()
    qcvars = record_dict["properties"]
    level_of_theory = f"{record_dict['specification']['levels'][2]['method']}/{record_dict['specification']['levels'][2]['basis']}"
    CP_IE = qcvars['results']['cp_corrected_interaction_energy'] * h2kcalmol
    NOCP_IE = qcvars['results'].get('nocp_corrected_interaction_energy', np.nan) * h2kcalmol
    return (
    record.initial_molecule,
    CP_IE,
    NOCP_IE,
    record.initial_molecule.atomic_numbers,
    record.initial_molecule.geometry * qcel.constants.bohr2angstroms,
    int(record.initial_molecule.molecular_charge),
    record.initial_molecule.molecular_multiplicity,
    )

def assemble_data_value_names():
    return [
        'qcel_molecule',
        "CP_IE",
        "NOCP_IE",
        "Z",
        "R",
        "TQ",
        "molecular_multiplicity"
    ]

df_mb = ds_mb.compile_values(
    value_call=assemble_data,
    value_names=assemble_data_value_names(),
    unpack=True,
)

pp(df_mb.columns.tolist())

In [ ]:
from cdsg_plot import error_statistics

df_sapt0['sapt0 total energes'] = df_sapt0['SAPT Energies'].apply(lambda x: x[0] * h2kcalmol)
df_plot = pd.DataFrame(
    {
        "qcel_molecule": df_mb["psi4/pbe/6-31g*"]["qcel_molecule"],
        "HF/6-31G*": df_mb["psi4/hf/6-31g*"]["CP_IE"],
        "PBE/6-31G*": df_mb["psi4/pbe/6-31g*"]["CP_IE"],
        'SAPT0/cc-pvdz': df_sapt0['sapt0 total energes'].values,
    }
)
print(df_plot)
id = [int(i[7:]) for i in df_plot.index]
df_plot['id'] = id
df_plot.sort_values(by='id', inplace=True, ascending=True)
df_plot['reference'] = ref_IEs
df_plot['SAPT0/aug-cc-pvdz'] = sapt0_adz
df_plot['HF/6-31G* error'] = (df_plot['HF/6-31G*'] - df_plot['reference']).astype(float)
df_plot['PBE/6-31G* error'] = (df_plot['PBE/6-31G*'] - df_plot['reference']).astype(float)
df_plot['SAPT0/cc-pvdz error'] = (df_plot['SAPT0/cc-pvdz'] - df_plot['reference']).astype(float)
df_plot['SAPT0/aug-cc-pvdz error'] = (df_plot['SAPT0/aug-cc-pvdz'] - df_plot['reference']).astype(float)
pd.set_option('display.max_rows', None)
print(df_plot[['PBE/6-31G*', 'SAPT0/cc-pvdz', 'reference', "SAPT0/aug-cc-pvdz"]])
print(df_plot[['PBE/6-31G* error', 'SAPT0/cc-pvdz error', "SAPT0/aug-cc-pvdz error"]].describe())

# Plotting the interaction energy errors

In [ ]:
error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        "PBE/6-31G*": "PBE/6-31G* error",
        # "B3LYP/6-31G*": "B3LYP/6-31G* error",
        "SAPT0/cc-pvdz": "SAPT0/cc-pvdz error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
    },
    output_filename="S22-IE.png",
    figure_size=(6, 6),
    x_label_fontsize=16,
    ylim=(-15, 15),
    rcParams={},
    usetex=False,
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

# QCMLForge

## AP-Net2 inference

In [ ]:
import apnet_pt
from apnet_pt.AtomPairwiseModels.apnet2 import APNet2Model
from apnet_pt.AtomModels.ap2_atom_model import AtomModel

atom_model = AtomModel().set_pretrained_model(model_id=0)
ap2 = APNet2Model(atom_model=atom_model.model).set_pretrained_model(model_id=0)
ap2.atom_model = atom_model.model
apnet2_ies_predicted = ap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=16
)

In [ ]:
# AP-Net2 IE
df_plot['APNet2'] = np.sum(apnet2_ies_predicted, axis=1)
df_plot['APNet2 error'] = (df_plot['APNet2'] - df_plot['reference']).astype(float)
print(df_plot.sort_values(by='APNet2 error', ascending=True)[['APNet2', 'reference']])
error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/cc-pvdz": "SAPT0/cc-pvdz error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
    },
    output_filename="S22-IE-AP2.png",
    rcParams={},
    usetex=False,
    figure_size=(4, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

In [ ]:
# Training models on new QM data: Transfer Learning

from apnet_pt import pairwise_datasets

ds2 = pairwise_datasets.apnet2_module_dataset(
    root="data_dir",
    spec_type=None,
    atom_model=atom_model,
    qcel_molecules=df_plot['qcel_molecule'].tolist(),
    energy_labels=[np.array([i]) for i in df_plot['reference'].tolist()],
    skip_compile=True,
    force_reprocess=True,
    atomic_batch_size=8,
    prebatched=False,
    in_memory=True,
    batch_size=4,
)
print(ds2)

## Transfer Learning

In [ ]:
# Transfer Learning APNet2 model on computed QM data
ap2.train(
    dataset=ds2,
    n_epochs=50,
    transfer_learning=True,
    skip_compile=True,
    model_path="apnet2_transfer_learning.pt",
    split_percent=0.8,
)

In [ ]:
# AP-Net2 IE
apnet2_ies_predicted_transfer = ap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=16,
)
df_plot['APNet2 transfer'] = np.sum(apnet2_ies_predicted_transfer, axis=1)
df_plot['APNet2 transfer error'] = (df_plot['APNet2 transfer'] - df_plot['reference']).astype(float)

error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
        "APNet2 transfer": "APNet2 transfer error",
    },
    output_filename="S22-IE-AP2.png",
    rcParams={},
    usetex=False,
    figure_size=(6, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

## $\Delta$AP-Net2

In [ ]:
from apnet_pt.pt_datasets.dapnet_ds import dapnet2_module_dataset_apnetStored

delta_energies = df_plot['HF/6-31G* error'].tolist()

# Only operates in pre-batched mode
ds = dapnet2_module_dataset_apnetStored(
    root="data_dir",
    r_cut=5.0,
    r_cut_im=8.0,
    spec_type=None,
    max_size=None,
    force_reprocess=True,
    batch_size=2,
    num_devices=1,
    skip_processed=False,
    skip_compile=True,
    print_level=2,
    in_memory=True,
    m1="HF/6-31G*",
    m2="CCSD(T)/CBS",
    qcel_molecules=df_plot['qcel_molecule'].tolist(),
    energy_labels=delta_energies,
)
print(ds)

In [ ]:
from apnet_pt.AtomPairwiseModels.dapnet2 import dAPNet2Model

dap2 = dAPNet2Model(
    atom_model=AtomModel().set_pretrained_model(model_id=0),
    apnet2_model=APNet2Model().set_pretrained_model(model_id=0).set_return_hidden_states(True),
)
dap2.train(
    ds,
    n_epochs=50,
    skip_compile=True,
    split_percent=0.6,
)

In [ ]:
dAPNet2_ies_predicted_transfer = dap2.predict_qcel_mols(
    mols=df_plot['qcel_molecule'].tolist(),
    batch_size=2,
)
df_plot['dAPNet2'] = dAPNet2_ies_predicted_transfer
df_plot['HF/6-31G*-dAPNet2'] = df_plot['HF/6-31G*'] - df_plot['dAPNet2']
print(df_plot[['dAPNet2', 'HF/6-31G*', 'HF/6-31G*-dAPNet2',  'reference']])
df_plot['dAPNet2 error'] = (df_plot['HF/6-31G*-dAPNet2'] - df_plot['reference']).astype(float)

error_statistics.violin_plot(
    df_plot,
    df_labels_and_columns={
        "HF/6-31G*": "HF/6-31G* error",
        "PBE/6-31G*": "PBE/6-31G* error",
        "SAPT0/aug-cc-pvdz": "SAPT0/aug-cc-pvdz error",
        "APNet2": "APNet2 error",
        "APNet2 transfer": "APNet2 transfer error",
        "dAPNet2 HF/6-31G* to CCSD(T)/CBS": "dAPNet2 error",
    },
    output_filename="S22-IE-AP2-dAP2.png",
    rcParams={},
    usetex=False,
    figure_size=(6, 4),
    ylabel=r"IE Error vs. CCSD(T)/CBS (kcal/mol)",
)

In [ ]:
# Be careful with this for it can corrupt running status...
# !ps aux | grep qcfractal | awk '{ print $2 }' | xargs kill -9

# The end...